In [1]:
import pandas as pd
import numpy as np

## read in the ISC Intrinsic parameters
* it was organized into different sheets

In [43]:
# read in the excel file

cam_intri_sheets = pd.read_excel('../datasets/Safety_Challenge_data/validation_data/calibration/CameraIntrinsicsv2.xlsx', sheet_name=None)
for sheet_name, sheet in cam_intri_sheets.items():
    print(sheet_name)
    # print(sheet)

VisualCamera1
VisualCamera2
VisualCamera3
VisualCamera4
VisualCamera5
VisualCamera6
VisualCamera7
VisualCamera8
ThermalCamera1
ThermalCamera2
ThermalCamera3
ThermalCamera4
ThermalCamera5


In [23]:
cam_intri_sheets['VisualCamera1']

,Intrinsic Matrix
0,"[1379.3550635419430818728869780898, ..."
1,"[ 0, 1378.17884..."
2,"[ 0, ..."
3,NaN
4,Radial Distortion
5,0.023226
6,-0.065994
7,0.033929
8,NaN
9,Tangential Distortion


In [24]:

cam_intri_sheets['VisualCamera1'].iloc[0,0]
# convert str '[num, num, num]' to list of numbers
cam_intri_sheets['VisualCamera1'].iloc[0,0].replace('[','').replace(']','').split(', ')
# convert list of str to list of float
np.array(cam_intri_sheets['VisualCamera1'].iloc[0,0].replace('[','').replace(']','').split(', ')).astype(float)

array([1379.35506354,    0.        ,  952.85219438])

In [52]:
def create_cam_intri(sheet_df):
    # get the K matrix
    # get the D matrix
    # each row of K
    K_0 = np.array(sheet_df.iloc[0,0].replace('[','').replace(']','').replace(' ', '').split(',')).astype(float)
    K_1 = np.array(sheet_df.iloc[1,0].replace('[','').replace(']','').replace(' ', '').split(',')).astype(float)
    K_2 = np.array(sheet_df.iloc[2,0].replace('[','').replace(']','').replace(' ', '').split(',')).astype(float)
    
    # stack the rows of K to get the K matrix
    K = np.stack([K_0, K_1, K_2], axis=0).ravel()

    # D = [k1, k2, p1, p2, k3] #k = radial distortion, p = tangential distortion
    k1 = sheet_df.iloc[5, 0]
    k2 = sheet_df.iloc[6, 0]
    k3 = sheet_df.iloc[7, 0]
    p1 = sheet_df.iloc[10, 0]
    p2 = sheet_df.iloc[11, 0]
    D = np.array([k1, k2, p1, p2, k3]).astype(float)
    return K, D
K, D = create_cam_intri(cam_intri_sheets['VisualCamera1'])
print(K, D)

[1.37935506e+03 0.00000000e+00 9.52852194e+02 0.00000000e+00
 1.37817884e+03 5.00982572e+02 0.00000000e+00 0.00000000e+00
 1.00000000e+00] [ 0.0232262  -0.06599417 -0.00877611 -0.00170374  0.0339294 ]


## read in the extrinsic calibration results
* The T matrix is already in the form of (4 * 4) transformation matrix [R | T] and the last row is 0, 0 , 0, 1

In [64]:
cam_extri_sheets = pd.read_excel('../datasets/Safety_Challenge_data/validation_data/calibration/CameraCameraExtrinsicCalibResultsv2.xlsx', sheet_name=None)
for sheet_name, sheet in cam_extri_sheets.items():
    print(sheet_name)
    # print(sheet)


VisualCamera1-VisualCamera5
VisualCamera2-VisualCamera4
VisualCamera2-VisualCamera7
VisualCamera3-VisualCamera5
VisualCamera4-VisualCamera5
VisualCamera4-VisualCamera6
VisualCamera4-VisualCamera7
VisualCamera5-VisualCamera6
VisualCamera5-VisualCamera8
VisualCamera6-VisualCamera7
VisualCamera7-VisualCamera8
VisualCamera8-VisualCamera4


In [65]:
cam_extri_sheets['VisualCamera1-VisualCamera5'].iloc[1,0].replace('[[','').replace('],','').split(', ')

[' 0.91108039', '-0.16562823', ' 0.37749147', '-7.69394633']

In [66]:
def create_cam_extri(sheet_name, sheet_df):
    # get the transformation matrix [R, T
    #                                0, 1]
    # get the T matrix
    # each row of R
    # replace all the white spaces and brackets

    T_0 = np.array(sheet_df.iloc[1,0].replace('[[','').replace('],','').replace(' ', '').split(',')).astype(float)
    T_1 = np.array(sheet_df.iloc[2,0].replace('[','').replace('],','').replace(' ', '').split(',')).astype(float)
    T_2 = np.array(sheet_df.iloc[3,0].replace('[','').replace('],','').replace(' ', '').split(',')).astype(float)
    T_3 = np.array(sheet_df.iloc[4,0].replace('[','').replace(']]','').replace(' ', '').split(',')).astype(float)
    
    # stack the rows of R to get the R matrix
    T = np.stack([T_0, T_1, T_2, T_3], axis=0).ravel()
    return T
T = create_cam_extri('VisualCamera2-VisualCamera4', cam_extri_sheets['VisualCamera2-VisualCamera4'])  
print(T)


[ 9.73425040e-01 -3.00101200e-02 -2.27031030e-01  1.75689994e+01
 -2.85214000e-02  9.67770360e-01 -2.50214040e-01  2.25364641e+01
  2.27222850e-01  2.50039850e-01  9.41195970e-01 -5.68831732e+01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]


## generate the cam_to_cam file
* It contains all intrinsic matrix for 9 cameras (8 Visual, 1 Thermal).
* Each matrix is in a row
* K: [fx 0 cx 0 fy cy 0 0 1]
* D: [k1, k2, p1, p2, k3]
* T: 4 * 4 transformation matrix

In [74]:
import os
def create_calib_file(cam_intri_sheets, cam_extri_sheets):
    output_path = '../datasets/Safety_Challenge_data/validation_data/calibration/'
    calib_file = open(os.path.join(output_path, 'cam_to_cam.txt'), 'w')
    for cam_name, cam_intri in cam_intri_sheets.items():
        try:
            K, D = create_cam_intri(cam_intri)
        except ValueError as e:
            print(e)
            print(f'Error in {cam_name}')
            break
        # write the K into a line
        K_string = ' '.join(map(str, K))
        D_string = ' '.join(map(str, D))
        calib_file.write(f'K_{cam_name}: {K_string}\n')
        calib_file.write(f'D_{cam_name}: {D_string}\n')
    for cam_pair, cam_extri in cam_extri_sheets.items():
        try:
            T = create_cam_extri(cam_pair, cam_extri)
        except ValueError as e:
            print(e)
            print(f'Error in {cam_pair}')
            break
        T_string = ' '.join(map(str, T))
        calib_file.write(f'T_{cam_pair}: {T_string}\n')
    calib_file.close()
create_calib_file(cam_intri_sheets, cam_extri_sheets)

## read in the calibration file to check everything is saved properly

In [76]:
# read the file to check if it is written correctly
calib_file = open('../datasets/Safety_Challenge_data/validation_data/calibration/cam_to_cam.txt', 'r')
print(calib_file.read().splitlines()[0])

K_VisualCamera1: 1379.355063541943 0.0 952.8521943785312 0.0 1378.1788420760524 500.98257206400126 0.0 0.0 1.0


In [ ]:
## Note: I added the lidar to Lidar manually to the cam_to_cam.txt file